In [1]:
import os
import torch
import time
import json
import csv
import shutil
import pandas as pd
import argparse
import jieba
jieba.load_userdict('dic.txt')
from openai import OpenAI
from utils import (read_docx_files, process_docx_files_2_para, process_docx_files_2_sents, process_docx_files_sents_2_longer, process_docx_files_year,
                   extract_context, clean_json_delimiters, process_docx_files_2_sents_WithNum, tender_document, tender_document_2_para, split_sentences)
from check_duplicate_in_sent import detect_duplicate_phrases, longest_string

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\admin\AppData\Local\Temp\jieba.cache
Loading model cost 0.386 seconds.
Prefix dict has been built successfully.


In [ ]:
from sentence_transformers.util import cos_sim  
from sentence_transformers import SentenceTransformer as SBert
model = SBert('paraphrase-multilingual-MiniLM-L12-v2')

In [11]:
version = 'final-1'

In [3]:
type1_1 = pd.read_excel(f'../answer_{version}-副本/answers_type1-常识错误-不未错误-备份.xlsx')
type1_1 = type1_1.drop_duplicates(subset=['id', 'sent_id'], keep='last').reset_index(drop=True)
white_list_id = ['新能源']
white_list_in = ['用人单位', '信用中国', '未划分标包']
type1_1 = type1_1[~type1_1['id'].str.contains('|'.join(white_list_id))]
type1_1 = type1_1[~type1_1['sent'].str.contains('|'.join(white_list_in))]

In [10]:
type1_2 = pd.read_excel(f'../answer_{version}-副本/answers_type1-常识错误-时间错误-OnlyTime-备份.xlsx')
white_list_id = ['审计法', '保险', '化工行业周报', '平安', '玉门油田', '电力', '西南']
white_list_in = ['招标文件', '5个工作日', '天然气', '投标文件递交', '估算金额', '网上开标', '招标文件的获取', '票房', '五一', 'Q1', '编号', '招标公告', '有效期', '货代', '预算管理', '电子', '央行', '23:59']
type1_2 = type1_2[~type1_2['id'].str.contains('|'.join(white_list_id))]
type1_2 = type1_2[~type1_2['sent'].str.contains('|'.join(white_list_in))]
type1_2 = type1_2.reset_index(drop=True)

In [12]:
type2 = pd.read_excel(f'../answer_{version}-副本/answers_type2-数值单位错误-备份.xlsx')
white_list_id = []
white_list_in = ['天然气', '吨', '国五', '重卡', 'MW', '标段', '结算', '截止日', '流入', 'PMI', '央行']
type1_2 = type1_2[~type1_2['id'].str.contains('|'.join(white_list_id))]
type1_2 = type1_2[~type1_2['sent'].str.contains('|'.join(white_list_in))]

In [13]:
type7 = pd.read_excel(f'../answer_{version}-副本/answers_type7-计算错误-备份.xlsx')

In [14]:
type8 = pd.read_excel(f'../answer_{version}-副本/answers_type8-语句机械+大模型-备份.xlsx')